In [1]:
import gi
gi.require_version('Gst', '1.0')
from gi.repository import Gst, GObject, GLib

def split_video_audio(input_file, audio_output_file, video_output_file):
    """
    Splits an MP4 video file into separate audio and video files using GStreamer.

    Args:
        input_file (str): Path to the input MP4 video file.
        audio_output_file (str): Path to the output audio file (e.g., "audio.mp3").  Must have a file extension suitable for encoding.
        video_output_file (str): Path to the output video file (e.g., "video.mp4"). Must have a file extension suitable for encoding.
    """

    Gst.init(None)

    # Create the pipeline
    pipeline = Gst.Pipeline()

    # Create elements
    source = Gst.ElementFactory.make("filesrc", "source")
    demuxer = Gst.ElementFactory.make("qtdemux", "demuxer") # For MP4 files
    audio_queue = Gst.ElementFactory.make("queue", "audio_queue")
    video_queue = Gst.ElementFactory.make("queue", "video_queue")

    # Audio elements
    audio_convert = Gst.ElementFactory.make("audioconvert", "audio_convert")
    audio_encoder = None # Choose an audio encoder based on output file extension
    if audio_output_file.endswith(".mp3"):
        audio_encoder = Gst.ElementFactory.make("lamemp3enc", "audio_encoder")
    elif audio_output_file.endswith(".ogg"):
        audio_encoder = Gst.ElementFactory.make("vorbisenc", "audio_encoder")
        audio_encoder.set_property("quality", 0.4)  # Adjust quality as needed
    elif audio_output_file.endswith(".aac"):
        audio_encoder = Gst.ElementFactory.make("faac", "audio_encoder")  # requires faac package.  Alternative: avenc_aac
        #audio_encoder = Gst.ElementFactory.make("avenc_aac", "audio_encoder")
    elif audio_output_file.endswith(".wav"):
        audio_encoder = Gst.ElementFactory.make("wavenc", "audio_encoder") # Not really an encoder but puts it in wave format
    else:
        raise ValueError("Unsupported audio output file extension.  Try .mp3, .ogg, .aac, or .wav")


    audio_sink = Gst.ElementFactory.make("filesink", "audio_sink")

    # Video elements
    video_convert = Gst.ElementFactory.make("videoconvert", "video_convert")
    video_encoder = None  # Choose a video encoder based on output file extension
    if video_output_file.endswith(".mp4"):
         video_encoder = Gst.ElementFactory.make("x264enc", "video_encoder")
         video_encoder.set_property("tune", "zerolatency") #For low latency encoding
         video_encoder.set_property("pass", 5)   # Increase quality, higher number better quality.  Defaults to 0
         video_encoder.set_property("bitrate", 2048) # Bitrate in kbps
         mp4mux = Gst.ElementFactory.make("mp4mux", "mp4mux")
    elif video_output_file.endswith(".webm"):
        video_encoder = Gst.ElementFactory.make("vp8enc", "video_encoder") #Or vp9enc for better encoding, but is more resource intensive
        muxer = Gst.ElementFactory.make("webmmux", "webmmux")
    elif video_output_file.endswith(".avi"):
         video_encoder = Gst.ElementFactory.make("x264enc", "video_encoder")
         muxer = Gst.ElementFactory.make("avimux", "avimux")

    else:
        raise ValueError("Unsupported video output file extension. Try .mp4, .webm or .avi")


    video_sink = Gst.ElementFactory.make("filesink", "video_sink")



    # Set properties
    source.set_property("location", input_file)
    audio_sink.set_property("location", audio_output_file)
    video_sink.set_property("location", video_output_file)



    # Add elements to the pipeline
    elements = [source, demuxer, audio_queue, video_queue, audio_convert, audio_encoder, audio_sink, video_convert, video_encoder, video_sink]
    if video_output_file.endswith(".mp4"):
        elements.append(mp4mux)
        pipeline.add(source, demuxer, audio_queue, video_queue, audio_convert, audio_encoder, audio_sink, video_convert, video_encoder, mp4mux, video_sink)
    else:
        elements.append(muxer)
        pipeline.add(source, demuxer, audio_queue, video_queue, audio_convert, audio_encoder, audio_sink, video_convert, video_encoder, muxer, video_sink)


    # Link elements
    source.link(demuxer)
    demuxer.connect("pad-added", on_pad_added, audio_queue, video_queue)

    #Audio branch
    audio_queue.link(audio_convert)
    audio_convert.link(audio_encoder)
    audio_encoder.link(audio_sink)

    #Video branch
    video_queue.link(video_convert)
    video_convert.link(video_encoder)
    if video_output_file.endswith(".mp4"):
        video_encoder.link(mp4mux)
        mp4mux.link(video_sink)
    else:
        video_encoder.link(muxer)
        muxer.link(video_sink)

    # Create bus and set callbacks
    bus = pipeline.get_bus()
    bus.add_signal_watch()
    bus.connect("message::eos", on_eos, pipeline)
    bus.connect("message::error", on_error, pipeline)

    # Start the pipeline
    pipeline.set_state(Gst.State.PLAYING)

    # Run the main loop
    loop = GLib.MainLoop()
    try:
        loop.run()
    except KeyboardInterrupt:
        print("Interrupted")
        pipeline.set_state(Gst.State.NULL)
        loop.quit()



def on_pad_added(demuxer, pad, audio_queue, video_queue):
    """
    Callback function for when a pad is added to the demuxer.  Connects the correct stream to the appropriate queue.
    """
    padname = pad.get_name()
    if padname.startswith("audio"):
        queue = audio_queue
    elif padname.startswith("video"):
        queue = video_queue
    else:
        print(f"Unknown pad {padname}, ignoring.")
        return

    sinkpad = queue.get_static_pad("sink")
    if sinkpad.is_linked():
        print(f"Pad {sinkpad.get_name()} is already linked, ignoring.")
        return

    new_pad_type = pad.query_caps(None).to_string()
    print(f"Linking demuxer pad {pad.get_name()} of type {new_pad_type} to queue {queue.get_name()}")

    ret = pad.link(sinkpad)
    if ret != Gst.PadLinkReturn.OK:
        print(f"Type is {new_pad_type}")
        print("Pad link failed:", ret)


def on_eos(bus, message, pipeline):
    """
    Callback function for when the end of stream is reached.
    """
    print("End of stream")
    pipeline.set_state(Gst.State.NULL)
    GLib.MainLoop().quit()

def on_error(bus, message, pipeline):
    """
    Callback function for when an error occurs.
    """
    err, debug_info = message.parse_error()
    print("Error received from element %s: %s" % (message.src.get_name(), err))
    print("Debugging information: %s" % debug_info)
    pipeline.set_state(Gst.State.NULL)
    GLib.MainLoop().quit()


if __name__ == '__main__':
    input_file = "input.mp4"  # Replace with your input file
    audio_output_file = "audio.mp3"  # Replace with your desired audio output file
    video_output_file = "video.mp4"  # Replace with your desired video output file

    # Create a dummy input file if it doesn't exist, to prevent crashes.
    import os
    if not os.path.exists(input_file):
        print(f"Warning:  Input file {input_file} does not exist.  Creating a dummy file.")
        open(input_file, 'a').close()


    try:
        split_video_audio(input_file, audio_output_file, video_output_file)
    except ValueError as e:
        print(f"Error: {e}")
        print("Please check your output file extensions and ensure the necessary codecs are installed.")


(process:34228): GLib-GIRepository-WARNING **: 21:31:28.747: Failed to load shared library 'libgobject-2.0.0.dylib' referenced by the typelib: dlopen(libgobject-2.0.0.dylib, 0x0009): tried: 'libgobject-2.0.0.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OSlibgobject-2.0.0.dylib' (no such file), '/usr/lib/libgobject-2.0.0.dylib' (no such file, not in dyld cache), 'libgobject-2.0.0.dylib' (no such file)

(process:34228): GLib-GIRepository-WARNING **: 21:31:28.747: Failed to load shared library 'libglib-2.0.0.dylib' referenced by the typelib: dlopen(libglib-2.0.0.dylib, 0x0009): tried: 'libglib-2.0.0.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OSlibglib-2.0.0.dylib' (no such file), '/usr/lib/libglib-2.0.0.dylib' (no such file, not in dyld cache), 'libglib-2.0.0.dylib' (no such file)


GError: gi-invoke-error-quark: Could not locate g_option_error_quark: dlopen(libglib-2.0.0.dylib, 0x0009): tried: 'libglib-2.0.0.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OSlibglib-2.0.0.dylib' (no such file), '/usr/lib/libglib-2.0.0.dylib' (no such file, not in dyld cache), 'libglib-2.0.0.dylib' (no such file) (1)